# Interpretability 

Having a model that you understand and trust is very important.  There is typically a tradeoff between performance of the model and interpretability 
![](https://miro.medium.com/max/1950/1*shNOspLyVn_2mvwves9MMA.png)
[Image Source](https://medium.com/ansaro-blog/interpreting-machine-learning-models-1234d735d6c9)

I am going to go through some techniques for interpretting your model - starting with linear regression.  

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# load in dataset 
df = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2021/train.csv')

df.head()

In [ ]:
# set id to be the index 
df.set_index('id', inplace = True)

In [ ]:
# begin with a train test split 
from sklearn.model_selection import train_test_split

X = df.drop('target', axis = 1)
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=11)

## Linear Regression

I am going to use the [statsmodels](https://www.statsmodels.org/stable/generated/statsmodels.regression.linear_model.OLS.html) library to run my regression.  I like to use statsmodels because it gives you access to a lot of extra information that sklearn does not provide natively.  

In [ ]:
import statsmodels.api as sm

# ad y intercept
X_train = sm.add_constant(X_train)

model = sm.OLS(y_train, X_train).fit()
model.summary()

I see that I have a $R^2$ value of 0.2 which indicates that I do not have a good fit, I have one feature that is not statistically significant at an alpha level 0f 0.05 `cont14`, and I have a Jarque-Bera value that is large indicating that my residuals are not normally distributed.  

> For the purposes of this notebook, I am just going to focus on interpreting the current model and not addresses any of the above issues.  

Now I want to see what the Root Mean Squarred Error is for this model 

In [ ]:
from sklearn.metrics import mean_squared_error

# add y_intercept to X_test
X_test = sm.add_constant(X_test)

np.sqrt(mean_squared_error(y_test, model.predict(X_test)))

## Global Interpretability 

To start I am going to look at some global interpretability techniques.  Global interpretability looks at how the model makes decisions in general.  

### Feature Importance 

The importance of a feature in a linear regression model can be measured by the absolute value of its t-statistic.
[Source](https://christophm.github.io/interpretable-ml-book/limo.html)

In [ ]:
import matplotlib.pyplot as plt 
import seaborn as sns 

In [ ]:
with plt.style.context('fivethirtyeight'):
    abs(model.tvalues)[1:].sort_values().plot(kind = 'barh', figsize = (8, 6))
    plt.title('Most Important Features According to Linear Regression')

We see that `cont10` is the feature that our model thinks is the most important feature 

### Weight Plot 

A weight plot shows the coefficients with the 95% confidence intervals for each coefficient

In [ ]:
# first make a dataframe with coefficents and 95% confidence interval 

wp = model.conf_int()
wp.columns = ['lower_bound', 'upper_bound']
wp['coefficient'] = model.params
wp.drop('const', inplace = True)
wp['error'] = np.abs(wp['coefficient'] - wp['upper_bound'])

wp.head()

In [ ]:
fig, ax = plt.subplots(figsize = (8, 8))
plt.errorbar(wp['coefficient'], range(len(wp)), fmt='o', color='black', xerr = wp['error'], 
            ecolor='lightgray', elinewidth=3, capsize=0)
plt.axvline(0, color = 'black', linestyle = '--', alpha = 0.8)
plt.title('Weight Plot')
plt.yticks(range(len(wp)), wp.index);

This shows which direction each feature impacts the model.  We see that `cont10` has a strong negative impact on the model and `cont6` has a strong positive impact on the model.  Something else of note - we see that `cont14` crosses over the 0 line, this is because the feature is not statistically significant.  

### Effect Plot

The weights of the linear regression model can be more meaningfully analyzed when they are multiplied by the actual feature values.

In [ ]:
# make new dataframe that multiples the coefficient by the raw values 
effect_plot_df = pd.DataFrame()

for i in wp.index:
    effect_plot_df[i] = wp.loc[i, 'coefficient'] * X_train[i]
effect_plot_df.head()

In [ ]:
with plt.style.context('fivethirtyeight'):
    fig, ax = plt.subplots(figsize = (12, 6))
    sns.boxplot(data=effect_plot_df, width=0.5, ax = ax)
    plt.xticks(rotation = 90)
    plt.axhline(0, linestyle = '--', color = 'red')
    plt.title('Effect Plot')

We see that `cont10`, `cont6`, and `cont1` all have the biggest range of possible effects.  This makes sense, when we looked at our feature importance plot these were our most important features. 

If we knew what these feature represented, we could use our understanding of the problem as a sanity check that our model is moving in the correct direction. 

## Local Interpretability

Local interpretability looks at why the model made a prediction for a specific instance

In [ ]:
# look at the prediction for this instance 
X_test.iloc[0][1:] 

In [ ]:
with plt.style.context('fivethirtyeight'):
    predicted_value = np.round(model.predict(X_test.iloc[0].values)[0], 2)
    fig, ax = plt.subplots(figsize = (12, 6))
    sns.boxplot(data=effect_plot_df, width=0.5, ax = ax, zorder = 1)
    plt.xticks(rotation = 90)
    plt.title(f'Predicted: {predicted_value} - Actual Value: {np.round(y_test.iloc[0], 2)}')
    plt.scatter(range(len(X_test.iloc[0][1:])), y = wp['coefficient'] * X_test.iloc[0][1:],
                marker = 'x', color = 'red', zorder = 2)

This shows were our data point that we made a prediction on, where it falls in the distribution of data that we trained on.  It can be easier to put this into context if we take the mean prediciton. 

In [ ]:
model.predict(X_test).mean()

Our data point that we predicted on is very similar to the mean.  This makes sense because our data point looks to be pretty close to the median point for most of the features.  We see that our it is a little higher than the median on `cont10`, but is lower than the median on `cont6`.  These are the two most important features and balance each other out.  


Next we will look at decision trees. 

## Decision Trees 


In [ ]:
from sklearn.tree import DecisionTreeRegressor

# make a model 
# setting max_depth to be 10 to prevent overfitting
dt = DecisionTreeRegressor(max_depth = 10)

# remove constant variable used for regression from X_train and X_test
X_train.drop(columns='const', inplace = True)
X_test.drop(columns='const', inplace = True)

# fit model 
dt.fit(X_train, y_train)

# check root mean squared error 
np.sqrt(mean_squared_error(y_test, dt.predict(X_test)))

Similar performance to the linear regression model 

## Global Interpretability 

### Feature Importance 

In [ ]:
with plt.style.context('fivethirtyeight'):
    features=df.columns
    importances = dt.feature_importances_
    indices = np.argsort(importances)

    fig, ax = plt.subplots(figsize = (12, 6))
    plt.title('Most Important Features According to Decision Tree')
    plt.barh(range(len(indices)), importances[indices], align='center')
    plt.yticks(range(len(indices)), features[indices])
    plt.xlabel('Relative Importance')

We see that the most important features differ from the linear regression 

### Visualize Decision Tree

Visualizing the decision tree can give you both a global perspective and a local perspective of our model.  I am only going to look at the first 3 layers of splits, otherwise it is too large to see

In [ ]:
from sklearn import tree

fig = plt.figure(figsize=(25,20))
_ = tree.plot_tree(dt, 
                   feature_names=X_train.columns,
                   filled=True, max_depth = 3)

Next up I'll use K-Nearest Neighbors 

## K-Nearest Neighbors 

With K-Nearest Neighbors there is not a native way to get global interpretability, so we'll look at local interpretabilty 

### Local Interpretability 

We'll look at a single prediction and look at the 5 nearest neighbors 

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

knn = KNeighborsRegressor()

knn.fit(X_train, y_train)

np.sqrt(mean_squared_error(y_test, knn.predict(X_test)))

In [ ]:
# point to explore 
X_test.iloc[0]

In [ ]:
# dataframe with nearest neighbors 
nn_df = X_train.iloc[knn.kneighbors(X_test.iloc[0].values.reshape(1, -1))[1][0]]
nn_df

In [ ]:
with plt.style.context('fivethirtyeight'):
    prediction = np.round(knn.predict(X_test.iloc[0].values.reshape(1, -1))[0], 2)
    fig, ax = plt.subplots(figsize = (8, 6))
    sns.boxplot(data = nn_df)
    plt.xticks(rotation = 90)
    plt.title(f'Predicted: {prediction} - Actual Value: {np.round(y_test.iloc[0], 2)}')
    plt.scatter(range(len(X_test.iloc[0])), y = X_test.iloc[1],
                    marker = 'x', color = 'red', zorder = 2)

Here we see how similar the point we are making a prediction on to the 5 nearest neighbors.  We see that a lot of the features have very different values than the nearest neighbors.  

As a summary on the model performance so far:

| Model  | RMSE  |  
|---|---|
| Linear Regression  | 0.73  |    
| Decision Tree  | 0.72  |    
|  K-Neareset Neighbors | 0.77  |   

## Random Forest 



In [ ]:
from sklearn.ensemble import RandomForestRegressor

# make instance of model
rf = RandomForestRegressor()

# fit model
rf.fit(X_train, y_train)

# check root mean squared error 
np.sqrt(mean_squared_error(y_test, rf.predict(X_test)))

### Global Interpretability 

Lets look at the most important features 

In [ ]:
with plt.style.context('fivethirtyeight'):
    features=df.columns
    importances = rf.feature_importances_
    indices = np.argsort(importances)

    fig, ax = plt.subplots(figsize = (12, 6))
    plt.title('Most Important Features According to Random Forest')
    plt.barh(range(len(indices)), importances[indices], align='center')
    plt.yticks(range(len(indices)), features[indices])
    plt.xlabel('Relative Importance')

## XGBoost 

In [ ]:
import xgboost as xg 

# Make instance of model
xgb_model = xg.XGBRegressor(objective ='reg:squarederror', 
                  n_estimators = 10, seed = 11) 
  
# Fit model 
xgb_model.fit(X_train, y_train) 

# Check RMSSE
np.sqrt(mean_squared_error(y_test, xgb_model.predict(X_test))) 

### Global Interpretability with Feature Importance 

In [ ]:
with plt.style.context('fivethirtyeight'):
    features=df.columns
    importances = xgb_model.feature_importances_
    indices = np.argsort(importances)

    fig, ax = plt.subplots(figsize = (12, 6))
    plt.title('Most Important Features According to XGBoost')
    plt.barh(range(len(indices)), importances[indices], align='center')
    plt.yticks(range(len(indices)), features[indices])
    plt.xlabel('Relative Importance')

Now I am going to compare the feature importance from each of the algorithms 

In [ ]:
# make dataframe with the feature importances 

rf_indices = np.argsort(rf.feature_importances_)
dt_indices = np.argsort(dt.feature_importances_)
xgb_indices = np.argsort(xgb_model.feature_importances_)

df_fi = pd.DataFrame(range(1, (len(features))), index = features[rf_indices[::-1]])
df_fi.columns = ['rf']
df_fi['dt'] = pd.DataFrame(range(1, (len(features))), index = features[dt_indices[::-1]])
df_fi['xgb'] = pd.DataFrame(range(1, (len(features))), index = features[xgb_indices[::-1]])
df_fi['lr'] = pd.DataFrame(range(1, (len(features))), 
             index = features[np.argsort(np.abs(model.tvalues[1:]).values)][::-1])
df_fi

In [ ]:
# plot most important features 
with plt.style.context('fivethirtyeight'):
    df_fi.loc[df.drop(columns = 'target').columns].plot(kind = 'bar', figsize = (12, 6))

We see that all the models think that `cont2` is an important feature.  All the models besides linear regression thinks that `cont3` is an important feature.  We also see that all the models think that `cont5` is not an important feature.  

## Next Steps 

We only used interpretability techniques that are native to the models.  We will next explore some interpretability techniques that are model agnostic.  

Part II can be seen [here](https://www.kaggle.com/jth359/deep-dive-into-ml-interpretability-part-ii)